In [1]:
# Dependencies and Setup
import pandas as pd

# Load files
school_data_to_load = "../schools_complete.csv"
student_data_to_load = "../students_complete.csv"

# Read School and Student Data File and store into Pandas Data Frames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
school_data_complete.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


In [2]:
#District Summary

#collecting summary stats
schools = school_data_complete['school_name'].unique()
num_schools = len(schools)
total_budget = school_data_complete['budget'].unique().sum()
total_students = len(school_data_complete)

avg_math = school_data_complete['math_score'].mean()
avg_read = school_data_complete['reading_score'].mean()


stu_pass_math = school_data_complete.loc[school_data_complete['math_score']>= 70, :]
math_pass_rate = (len(stu_pass_math)/total_students)*100
stu_pass_read = school_data_complete.loc[school_data_complete['reading_score']>= 70, :]
read_pass_rate = (len(stu_pass_read)/total_students)*100
avg_pass_rate = (math_pass_rate + read_pass_rate)/2

#creating a df of summary stats through passing in a dictionary
district_summary = pd.DataFrame({"Number of Schools": num_schools, 
                               "Number of Students": [total_students],
                               "Total Budget": [total_budget],
                               "Average Math Score": [avg_math],
                               "Average Reading Score": [avg_read],
                               "Overall Passing Rate": [avg_pass_rate],
                               "% Students Passing Math": [math_pass_rate],
                               "% Students Passing Reading":[read_pass_rate] 
                                }
                               )

#formating the summary to look more readable
district_summary["Total Budget"] = district_summary["Total Budget"].map("${:,}".format)
district_summary["Number of Students"] = district_summary["Number of Students"].map("{:,}".format)
district_summary["Average Math Score"] = district_summary["Average Math Score"].map("{:.2f}%".format)
district_summary["Average Reading Score"] = district_summary["Average Reading Score"].map("{:.2f}%".format)
district_summary["Overall Passing Rate"] = district_summary["Overall Passing Rate"].map("{:.2f}%".format)
district_summary["% Students Passing Math"] = district_summary["% Students Passing Math"].map("{:.2f}%".format)
district_summary["% Students Passing Reading"] = district_summary["% Students Passing Reading"].map("{:.2f}%".format)


district_summary

,Number of Schools,Number of Students,Total Budget,Average Math Score,Average Reading Score,Overall Passing Rate,% Students Passing Math,% Students Passing Reading
0,15,"39,170","$24,649,428",78.99%,81.88%,80.39%,74.98%,85.81%


In [3]:
#School Summary

#grouping the dataframe by schools to summarize stats by school
school_group = school_data_complete.groupby(["school_name"])
school_students = school_group['student_name'].count()
school_summary = pd.DataFrame(school_students)
school_summary =school_summary.rename(columns = {'student_name': 'Total Students'})

school_summary["Average Math Score"] = school_group['math_score'].mean()
school_summary["Average Reading Score"] = school_group['reading_score'].mean()

# % passing math/reading is found by grouping the dfs that were filtered by math/reading >70 then dividing by the student count
passing_math = ((stu_pass_math.groupby(["school_name"])["Student ID"].count())/school_students)*100
school_summary["% Passing Math"]= passing_math
passing_reading = ((stu_pass_read.groupby(["school_name"])["Student ID"].count())/school_students)*100
school_summary["% Passing Reading"]= passing_reading
school_summary["% Overall Passing"] = (passing_math + passing_reading)/2

#Merging school data containing the school budget and types with the school summary data frame
school_data_bt = school_data.loc[:,["school_name","type", "budget"]]
school_summary =pd.merge(school_summary, school_data_bt, how="left", on=["school_name"])
school_summary["Per Student Budget"] = school_summary["budget"]/school_summary["Total Students"]


#Formating the data
column_order = ['school_name','type', 'budget', 'Per Student Budget', 'Total Students', 'Average Math Score', 'Average Reading Score', '% Passing Math', '% Passing Reading', '% Overall Passing']
school_summary = school_summary[column_order]
school_summary =school_summary.rename(columns = {'type': 'Type',
                                                'budget': 'Budget',
                                                'school_name': 'School Name'})

school_summary = school_summary.set_index('School Name')

school_summary

,Type,Budget,Per Student Budget,Total Students,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Name,,,,,,,,,
Bailey High School,District,3124928,628.0,4976,77.048432,81.033963,66.680064,81.933280,74.306672
Cabrera High School,Charter,1081356,582.0,1858,83.061895,83.975780,94.133477,97.039828,95.586652
Figueroa High School,District,1884411,639.0,2949,76.711767,81.158020,65.988471,80.739234,73.363852
Ford High School,District,1763916,644.0,2739,77.102592,80.746258,68.309602,79.299014,73.804308
Griffin High School,Charter,917500,625.0,1468,83.351499,83.816757,93.392371,97.138965,95.265668
Hernandez High School,District,3022020,652.0,4635,77.289752,80.934412,66.752967,80.862999,73.807983
Holden High School,Charter,248087,581.0,427,83.803279,83.814988,92.505855,96.252927,94.379391
Huang High School,District,1910635,655.0,2917,76.629414,81.182722,65.683922,81.316421,73.500171
Johnson High School,District,3094650,650.0,4761,77.072464,80.966394,66.057551,81.222432,73.639992


In [12]:
#Top Performing Schools (By Passing Rate)

school_summary.sort_values(by='% Overall Passing', ascending=False).head(5)

,Type,Budget,Per Student Budget,Total Students,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing,Per Student Spending Range,Number of students passing math,Number of students passing reading,School Size
School Name,,,,,,,,,,,,,
Cabrera High School,Charter,1081356,582.0,1858,83.061895,83.975780,94.133477,97.039828,95.586652,<$585,1749.0,1803.0,Medium (1000-2000)
Thomas High School,Charter,1043130,638.0,1635,83.418349,83.848930,93.272171,97.308869,95.290520,$615-645,1525.0,1591.0,Medium (1000-2000)
Pena High School,Charter,585858,609.0,962,83.839917,84.044699,94.594595,95.945946,95.270270,$585-615,910.0,923.0,Small (<1000)
Griffin High School,Charter,917500,625.0,1468,83.351499,83.816757,93.392371,97.138965,95.265668,$615-645,1371.0,1426.0,Medium (1000-2000)
Wilson High School,Charter,1319574,578.0,2283,83.274201,83.989488,93.867718,96.539641,95.203679,<$585,2143.0,2204.0,Large (2000-5000)


In [5]:
#Bottom Performing Schools (By Passing Rate)

school_summary.sort_values(by='% Overall Passing', ascending=True).head(5)

,Type,Budget,Per Student Budget,Total Students,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Name,,,,,,,,,
Rodriguez High School,District,2547363,637.0,3999,76.842711,80.744686,66.366592,80.220055,73.293323
Figueroa High School,District,1884411,639.0,2949,76.711767,81.158020,65.988471,80.739234,73.363852
Huang High School,District,1910635,655.0,2917,76.629414,81.182722,65.683922,81.316421,73.500171
Johnson High School,District,3094650,650.0,4761,77.072464,80.966394,66.057551,81.222432,73.639992
Ford High School,District,1763916,644.0,2739,77.102592,80.746258,68.309602,79.299014,73.804308


In [6]:
#Math Scores by Grade

#collect grade names into a list to be referenced
grades = school_data_complete['grade'].unique()

#creating series by filtering data by each grade then grouping by school. Then taking the mean of the math scores.
grade_a = school_data_complete.loc[school_data_complete['grade']== grades[0], :].groupby(['school_name'])['math_score'].mean()
grade_b = school_data_complete.loc[school_data_complete['grade']== grades[1], :].groupby(['school_name'])['math_score'].mean()
grade_c = school_data_complete.loc[school_data_complete['grade']== grades[2], :].groupby(['school_name'])['math_score'].mean()
grade_d = school_data_complete.loc[school_data_complete['grade']== grades[3], :].groupby(['school_name'])['math_score'].mean()

#Creating a dataframe with the grades series grouped by school
grade_summary_m = pd.DataFrame(grade_a)
grade_summary_m = grade_summary_m.rename(columns = {'math_score': grades[0]})
grade_summary_m[grades[3]] = grade_d
grade_summary_m[grades[2]] = grade_c
grade_summary_m[grades[1]] = grade_b

grade_summary_m

,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


In [7]:
#Reading Score by Grade

#Creating a series by filtering data by each grade then grouping by school. Then taking the mean of the reading scores.
grade_a_r = school_data_complete.loc[school_data_complete['grade']== grades[0], :].groupby(['school_name'])['reading_score'].mean()
grade_b_r = school_data_complete.loc[school_data_complete['grade']== grades[1], :].groupby(['school_name'])['reading_score'].mean()
grade_c_r = school_data_complete.loc[school_data_complete['grade']== grades[2], :].groupby(['school_name'])['reading_score'].mean()
grade_d_r = school_data_complete.loc[school_data_complete['grade']== grades[3], :].groupby(['school_name'])['reading_score'].mean()

#Creating a dataframe with the grades series grouped by school. 
grade_summary_r = pd.DataFrame(grade_a_r)
grade_summary_r = grade_summary_r.rename(columns = {'reading_score': grades[0]})
grade_summary_r[grades[3]] = grade_d_r
grade_summary_r[grades[2]] = grade_c_r
grade_summary_r[grades[1]] = grade_b_r

grade_summary_r

,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


In [8]:
#Scores by School Spending

# Bins Created for spending by student
spending_bins = [0, 585, 615, 645, 675]
group_names = ["<$585", "$585-615", "$615-645", "$645-675"]

school_summary["Per Student Spending Range"] = pd.cut(school_summary["Per Student Budget"], spending_bins, labels=group_names)

#Create new columns that total up number of students passing math/reading 
school_summary["Number of students passing math"] = school_summary['Total Students']*(school_summary['% Passing Math']/100)
school_summary["Number of students passing reading"] = school_summary['Total Students']*(school_summary['% Passing Reading']/100)

# grouby the df by speding range. Take the mean of average math/reading and take the sums of students counts
school_spending = school_summary[['Per Student Spending Range','Total Students','Average Math Score', 'Average Reading Score', 'Number of students passing math', 'Number of students passing reading']].groupby('Per Student Spending Range').agg(
    {'Average Math Score': 'mean',
    'Average Reading Score': 'mean',
    'Total Students': 'sum',
    'Number of students passing math': 'sum',
    'Number of students passing reading': 'sum'})
 
#Use the students counts of passing math/reading divided by the total students to get the percent passing each
school_spending['% Passing Math'] = (school_spending['Number of students passing math']/ school_spending['Total Students'])*100
school_spending['% Passing Reading'] = (school_spending['Number of students passing reading']/ school_spending['Total Students'])*100
school_spending['% Overall Passing Rate'] = ((school_spending['% Passing Math']+ school_spending['% Passing Reading'])/2)

#Selecting the columns to display
school_spending = school_spending[[ 'Average Math Score', 'Average Reading Score', '% Passing Math', '% Passing Reading', '% Overall Passing Rate']]
school_spending

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Per Student Spending Range,,,,,
<$585,83.455399,83.933814,93.702889,96.686558,95.194724
$585-615,83.599686,83.885211,94.124128,95.886889,95.005509
$615-645,79.079225,81.891436,71.400428,83.614770,77.507599
$645-675,76.997210,81.027843,66.230813,81.109397,73.670105


In [9]:
#Scores by School Size

# Bins created for school size by total students
size_bins = [0, 1000, 2000, 5000]
group_names_size = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

school_summary["School Size"] = pd.cut(school_summary["Total Students"], size_bins, labels=group_names_size)

#Create a df grouped by school size. Take the mean of average math/reading and take the sums of students counts
school_size = school_summary[['School Size','Total Students','Average Math Score', 'Average Reading Score', 'Number of students passing math', 'Number of students passing reading']].groupby('School Size').agg(
    {'Average Math Score': 'mean',
    'Average Reading Score': 'mean',
    'Total Students': 'sum',
    'Number of students passing math': 'sum',
    'Number of students passing reading': 'sum'})

#Use the students counts of passing math/reading divided by the total students to get the percent passing each
school_size['% Passing Math'] = (school_size['Number of students passing math']/ school_size['Total Students'])*100
school_size['% Passing Reading'] = (school_size['Number of students passing reading']/ school_size['Total Students'])*100
school_size['% Overall Passing Rate'] = ((school_size['% Passing Math']+ school_size['% Passing Reading'])/2)

#Selecting the columns to display
school_size = school_size[[ 'Average Math Score', 'Average Reading Score', '% Passing Math', '% Passing Reading', '% Overall Passing Rate']]

school_size

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Size,,,,,
Small (<1000),83.821598,83.929843,93.952484,96.040317,94.996400
Medium (1000-2000),83.374684,83.864438,93.616522,96.773058,95.194790
Large (2000-5000),77.746417,81.344493,68.652380,82.125158,75.388769


In [10]:
#Scores by School Type

type_group = school_data_complete.groupby(["type"])
school_students_type = type_group['student_name'].count()

#Dataframe created with the averages of math/reading scores grouped by type
type_summary = type_group['math_score', 'reading_score'].mean()

type_summary =type_summary.rename(columns = {'math_score': 'Average Math Score',
                                            'reading_score': 'Average Reading Score'})

#Group the dataframes containing only students passing math/reading by type and then divide by student count
passing_math_type = ((stu_pass_math.groupby(["type"])["Student ID"].count())/school_students_type)*100
passing_read_type = ((stu_pass_read.groupby(["type"])["Student ID"].count())/school_students_type)*100


#Adding new columns into the dataframe containing percents passing
type_summary['% Passing Math'] = passing_math_type
type_summary['% Passing Reading'] = passing_read_type
type_summary['% Overall Passing Rate'] = (passing_math_type + passing_read_type)/2


type_summary

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
type,,,,,
Charter,83.406183,83.902821,93.701821,96.645891,95.173856
District,76.987026,80.962485,66.518387,80.905249,73.711818
